# Spell Correction Program
This notebook demonstrates a spell correction application using two different approaches: SymSpell and SpellChecker. It includes a graphical user interface (GUI) built with `tkinter`.

## Imports
We start by importing the necessary libraries.

In [1]:
from tkinter import Tk, Label, Entry, Button, Text, Scrollbar, END
from symspellpy import SymSpell, Verbosity
from spellchecker import SpellChecker

## SymSpell Spell Correction
This function uses the SymSpell library to correct spellings.

In [2]:
def correct_spellings_symspell():
    text = text_entry.get()
    
    # Initialize SymSpell
    max_edit_distance_dictionary = 2
    prefix_length = 7
    sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
    
    # Load dictionary
    dictionary_path = "frequency_dictionary_en_82_765.txt"
    term_index = 0
    count_index = 1
    if not sym_spell.load_dictionary(dictionary_path, term_index, count_index):
        result_label.config(text="Dictionary file not found")
        return
    
    # Lookup suggestions
    max_edit_distance_lookup = 2
    suggestions = sym_spell.lookup_compound(text, max_edit_distance_lookup)
    corrected_text = " ".join(suggestion.term for suggestion in suggestions)
    result_label.config(text=corrected_text)
    
    # Update history
    if text.lower() == 'clear':
        history_text.config(state='normal')
        history_text.delete('1.0', END)
        history_text.config(state='disabled')
    else:
        history_text.config(state='normal')
        history_text.insert(END, f"Original: {text}\nCorrected (SymSpell): {corrected_text}\n\n")
        history_text.config(state='disabled')

## SpellChecker Spell Correction
This function uses the SpellChecker library to correct spellings.

In [3]:
def correct_spellings_spellchecker():
    text = text_entry.get()
    spell = SpellChecker()
    words = text.split()
    corrected_words = [spell.correction(word) for word in words]
    corrected_text = " ".join(corrected_words)
    result_label.config(text=f"Corrected Text: {corrected_text}")

    # Update history
    if text.lower() == 'clear':
        history_text.config(state='normal')
        history_text.delete('1.0', END)
        history_text.config(state='disabled')
    else:
        history_text.config(state='normal')
        history_text.insert(END, f"Original: {text}\nCorrected (SpellChecker): {corrected_text}\n\n")
        history_text.config(state='disabled')

## Clear History
This function clears the history text box.

In [4]:
def clear_history():
    history_text.config(state='normal')
    history_text.delete('1.0', END)
    history_text.config(state='disabled')

## Exit Program
This function exits the application.

In [5]:
def exit_program():
    root.destroy()

## GUI Setup
This function sets up the graphical user interface (GUI) for the spell correction program.

In [6]:
def setup_gui():
    global text_entry, result_label, history_text, root
    
    root = Tk()
    root.title("Spell Correction Program")
    
    text_label = Label(root, text="Enter text:")
    text_label.pack(pady=5)
    
    text_entry = Entry(root, width=30)
    text_entry.pack(pady=5)
    
    correct_button_symspell = Button(root, text="Correct Spelling (SymSpell)", command=correct_spellings_symspell)
    correct_button_symspell.pack(pady=5)
    
    correct_button_spellchecker = Button(root, text="Correct Spelling (SpellChecker)", command=correct_spellings_spellchecker)
    correct_button_spellchecker.pack(pady=5)
    
    result_label = Label(root, text="")
    result_label.pack(pady=5)
    
    history_label = Label(root, text="History:")
    history_label.pack(pady=5)
    
    history_text = Text(root, height=10, width=40)
    history_text.pack(pady=5)
    
    scrollbar = Scrollbar(root)
    scrollbar.pack(side='right', fill='y')
    history_text.config(yscrollcommand=scrollbar.set)
    scrollbar.config(command=history_text.yview)
    
    clear_button = Button(root, text="Clear History", command=clear_history)
    clear_button.pack(pady=5)
    
    exit_button = Button(root, text="Exit", command=exit_program)
    exit_button.pack(pady=5)
    
    root.mainloop()

## Main Execution
This code sets up and runs the GUI.

In [7]:
if __name__ == "__main__":
    setup_gui()

# Creating a dictionary from text and spell corrector

In [30]:
# Import the libraries for processing data and other utilities

import os, sys, gc, warnings
import logging, math, re, heapq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
from collections import Counter    
from nltk.tokenize import word_tokenize

# These settings help in proper formatting and display of the output of code we run
warnings.filterwarnings("ignore")
#InteractiveShell.ast_node_interactivity = "all"
# display(HTML(data="""<style> div#notebook-container{width:95%;}</style>"""))

# This is to download stop words for cleaning the tweets
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

In [14]:
# Load the dataset (Shakespeare's works)
input_file_location = 't8.shakespeare.txt'


In [15]:

# Read and preprocess the data
with open(input_file_location, 'r') as file:
    text = file.read().lower()
    word_list = re.findall(r'\w+', text)

In [16]:


# Create a vocabulary set and word frequency dictionary
vocab = set(word_list)
word_count_dict = Counter(word_list)
total_words = sum(word_count_dict.values())

# Calculate word probabilities
probs = {word: word_count / total_words for word, word_count in word_count_dict.items()}


In [17]:
def delete_letter(word):
    return [word[:i] + word[i+1:] for i in range(len(word))]

def switch_letter(word):
    return [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word) - 1)]

def replace_letter(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    return [word[:i] + l + word[i+1:] for i in range(len(word)) for l in letters]

def insert_letter(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    return [word[:i] + l + word[i:] for i in range(len(word) + 1) for l in letters]

def edit_one_letter(word, allow_switches=True):
    edits = set(delete_letter(word) + insert_letter(word) + replace_letter(word))
    if allow_switches:
        edits.update(switch_letter(word))
    return edits

def edit_two_letters(word, allow_switches=True):
    return set(e2 for e1 in edit_one_letter(word, allow_switches) for e2 in edit_one_letter(e1, allow_switches))


In [18]:
def get_spelling_suggestions(word, probs, vocab, n=2):
    suggestions = list((word in vocab and [word]) or 
                       edit_one_letter(word).intersection(vocab) or 
                       edit_two_letters(word).intersection(vocab))
    return sorted([(s, probs.get(s, 0)) for s in suggestions], key=lambda x: x[1], reverse=True)[:n]


In [19]:
# Example words to correct
my_words = ['dys','furthar','mercuryn','disdaain','tumtultous']

# Get corrections
for word in my_words:
    suggestions = get_spelling_suggestions(word, probs, vocab, 3)
    print(f"\nWord: {word}")
    for suggestion, prob in suggestions:
        print(f"Suggested: {suggestion}, Probability: {prob:.6f}")



Word: dys
Suggested: days, Probability: 0.000225
Suggested: dye, Probability: 0.000005
Suggested: dis, Probability: 0.000005

Word: furthar
Suggested: further, Probability: 0.000204

Word: mercuryn
Suggested: mercury, Probability: 0.000016

Word: disdaain
Suggested: disdain, Probability: 0.000042

Word: tumtultous
Suggested: tumultuous, Probability: 0.000004


In [20]:
def correct_sentence(sentence, probs, vocab):
    words = word_tokenize(sentence.lower())
    corrected_sentence = []
    for word in words:
        if word in vocab:
            corrected_sentence.append(word)
        else:
            suggestions = get_spelling_suggestions(word, probs, vocab, 1)
            if suggestions:
                corrected_sentence.append(suggestions[0][0])
            else:
                corrected_sentence.append(word)
    return ' '.join(corrected_sentence)




In [21]:
# Example usage
sentence = "Thiz is an exmple"
corrected_sentence = correct_sentence(sentence, probs, vocab)
print(f"Original sentence: {sentence}")
print(f"Corrected sentence: {corrected_sentence}")

Original sentence: Thiz is an exmple
Corrected sentence: this is an example


# Contextual corrector with build model

In [ ]:
# Download the spaCy model if not already present
#import spacy
#try:
    #nlp = spacy.load('en_core_web_sm')
#except OSError:
    #!python -m spacy download en_core_web_sm
    #nlp = spacy.load('en_core_web_sm')

In [29]:

# Download the spaCy model if not already present
import spacy
nlp = spacy.load('en_core_web_sm')

# Add contextual spell checker to the pipeline
import contextualSpellCheck
contextualSpellCheck.add_to_pipe(nlp)

# Create a document using the NLP model
doc = nlp('I came home so that as I would rather participate in the function the next dys.')

# Print the number of corrections in the input text
print(len(doc._.suggestions_spellCheck))

# Print all the actual corrections that were made with the associated mapping
print(doc._.suggestions_spellCheck)

# Display the outcome after spelling correction
print(doc._.outcome_spellCheck)


1
{dys: 'day'}
I came home so that as I would rather participate in the function the next day.


In [26]:
# Create a document using the NLP model
doc = nlp(' helo how ae you today.')

# Print the number of corrections in the input text
print(len(doc._.suggestions_spellCheck))

# Print all the actual corrections that were made with the associated mapping
print(doc._.suggestions_spellCheck)

# Display the outcome after spelling correction
print(doc._.outcome_spellCheck)

2
{hellow: 'Well', ae: 'are'}
 Well how are you today.
